In [1]:
#Download the content
import requests
#Parse 
from bs4 import BeautifulSoup
#Catch the content
from lxml import etree
#Extract content
import regex as re
#Lib spacy
import spacy
#Lib json
import json

In [23]:
# TODO: remover código não utilizado
# TODO: testar com outra norma, como http://www.planalto.gov.br/ccivil_03/leis/l8112cons.htm

RE_NORMALIZE_BLANKS = re.compile(r'[\s]+')
url = "http://www.planalto.gov.br/ccivil_03/constituicao/ConstituicaoCompilado.htm"
REQUEST_HEADER = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

def pprint(dados):
    print(json.dumps(dados, indent=2, ensure_ascii=False).encode('utf-8').decode())

#Clean the text
def replace_all_problems(text):
    fim = text.replace("\r\n", " ")
    return fim

#Normalize the text
def normalize_text(s):

    return RE_NORMALIZE_BLANKS.sub(' ', s).strip()

#Request the html
def download_html(url):
    r = requests.get(url, headers=REQUEST_HEADER)

    return r.text

#Split the html in the end of art 250
def remove_pos_normas_centrais(html):

    return html.split("Brasília, 5 de outubro de 1988.")[0]

#Parse the soup
def parse_soup(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    return soup

#Extract the preambulo
def extract_preambulo(soup):
    dom = etree.HTML(str(soup))
    content = replace_all_problems((dom.xpath('/html/body/p[3]/font/small')[0].text))
    dictionary = {"preambulo": str(content)}
    return dictionary

def create_title(soup):
    find = soup.find_all("p", face="Arial")
    return find

html = download_html(url)
html_preambulo_e_normas_centrais = remove_pos_normas_centrais(html)
soup = parse_soup(html_preambulo_e_normas_centrais)

cf = ' '.join(p.text for p in soup.find_all('p'))
cf = normalize_text(cf)
cf = cf.strip()

In [17]:
cf[:2000]

'Presidência da República Casa Civil Subchefia para Assuntos Jurídicos CONSTITUIÇÃO DA REPÚBLICA FEDERATIVA DO BRASIL DE 1988 Vide Emenda Constitucional nº 91, de 2016 Vide Emenda Constitucional nº 106, de 2020 Vide Emenda Constitucional nº 107, de 2020 Vide Emenda Constitucional nº 106, de 2020 Vide Emenda Constitucional nº 107, de 2020 Vide Emenda Constitucional nº 107, de 2020 Emendas Constitucionais Emendas Constitucionais de Revisão Ato das Disposições Constitucionais Transitórias Atos decorrentes do disposto no § 3º do art. 5º ÍNDICE TEMÁTICO PREÂMBULO Nós, representantes do povo brasileiro, reunidos em Assembléia Nacional Constituinte para instituir um Estado Democrático, destinado a assegurar o exercício dos direitos sociais e individuais, a liberdade, a segurança, o bem-estar, o desenvolvimento, a igualdade e a justiça como valores supremos de uma sociedade fraterna, pluralista e sem preconceitos, fundada na harmonia social e comprometida, na ordem interna e internacional, com

In [65]:
# ler sobre r-string 
# https://pt.stackoverflow.com/questions/80545/qual-%C3%A9-a-diferen%C3%A7a-entre-string-e-rstring-em-python

FIND_TITLE = re.compile(r'TÍTULO\s+(?P<identificador>[IXV]+)(?P<descricao>.*?)(?=Art\.|CAPÍTULO)(?P<conteudo>.*?)(?=TÍTULO|$)')
FIND_CHAPTER = re.compile(r'(CAPÍTULO\ (?P<identificador>[IXV]+))?(?P<descricao>.*?)(?=Art\.)(?P<conteudo>.*?)(?=CAPÍTULO|$)')
FIND_ARTICLE = re.compile(r'Art\.\ (?P<identificador>\d+)º?(?P<texto>.*?)(?=[IXV]+\ -\ )(?P<conteudo>.*?)(?=Art\.|$)')
FIND_INCISO = re.compile(r'(?P<identificador>[IXV]+)\ -\ (?P<texto>.*?)(?=[a-z]\)|[IXV]+\ -\ |$)(?P<conteudo>.*?)(?=[IXV]+\ -\ |$)')
FIND_ALINEA = re.compile(r'(?P<identificador>[a-z])\)(?P<texto>.*?)(?=[a-z]\)|$)')

# dar uma olhada no google python style guide 
# https://google.github.io/styleguide/pyguide.html

# cria o dicionário contendo o documento estruturado
dados = {'titulos': []}

# itera nos matches de seções do tipo título
for match_titulo in FIND_TITLE.finditer(cf):
    
    # instancia dicionário para armazenar os dados do título corrente
    titulo = {
        # recupera o identificador do título
        'identificador': match_titulo.group('identificador'), 
        # recupera a descrição do título
        'descricao': normalize_text(match_titulo.group('descricao')),
        # inicializa os capítulos do título
        'capitulos': []
    }
    
    # adiciona o título extraído aos dados
    dados['titulos'].append(titulo)
    
    # extrai o conteúdo do título
    titulo_conteudo = match_titulo.group('conteudo')
    
    # itera nos matches de seções do tipo capítulo
    for match_capitulo in FIND_CHAPTER.finditer(titulo_conteudo):
        
        # O RESTO SEGUE A MESMA LÓGICA -> ÓTIMO PARA REFATORAR ELIMINANDO REPETIÇÃO DE CÓDIGO
        
        capitulo = {
            'identificador': match_capitulo.group('identificador') or '0',
            'descricao': normalize_text(match_capitulo.group('descricao') or 'CAPÍTULO ÚNICO'),
            'artigos': []
        }
        
        titulo['capitulos'].append(capitulo)
        
        capitulo_conteudo = match_capitulo.group('conteudo')
        
        for match_artigo in FIND_ARTICLE.finditer(capitulo_conteudo):
            
            artigo = {
                'identificador': match_artigo.group('identificador'),
                'texto': normalize_text(match_artigo.group('texto')),
                'incisos': []
            }
            
            capitulo['artigos'].append(artigo)
            
            artigo_conteudo = match_artigo.group('conteudo')
            
            for match_inciso in FIND_INCISO.finditer(artigo_conteudo):
                
                inciso = {
                    'identificador': match_inciso.group('identificador'),
                    'texto': normalize_text(match_inciso.group('texto')),
                    'alineas': []
                }
                
                artigo['incisos'].append(inciso)
                
                inciso_conteudo = match_inciso.group('conteudo')
        
                for match_alinea in FIND_ALINEA.finditer(inciso_conteudo):
                
                    alinea = {
                        'identificador': match_alinea.group('identificador'),
                        'texto': normalize_text(match_alinea.group('texto'))
                    }
                    
                    inciso['alineas'].append(alinea)

In [64]:
pprint(dados['titulos'][1]['capitulos'][0]['artigos'][0]['incisos'][27])

{
  "identificador": "XXVIII",
  "texto": "são assegurados, nos termos da lei:",
  "alineas": [
    {
      "identificador": "a",
      "texto": " a proteção às participações individuais em obras coletivas e à reprodução da imagem e voz humanas, inclusive nas atividades desportivas; "
    },
    {
      "identificador": "b",
      "texto": " o direito de fiscalização do aproveitamento econômico das obras que criarem ou de que participarem aos criadores, aos intérpretes e às respectivas representações sindicais e associativas; "
    }
  ]
}


In [66]:
pprint(dados)

{
  "titulos": [
    {
      "identificador": "I",
      "descricao": "Dos Princípios Fundamentais",
      "capitulos": [
        {
          "identificador": "0",
          "descricao": "CAPÍTULO ÚNICO",
          "artigos": [
            {
              "identificador": "1",
              "texto": "A República Federativa do Brasil, formada pela união indissolúvel dos Estados e Municípios e do Distrito Federal, constitui-se em Estado Democrático de Direito e tem como fundamentos:",
              "incisos": [
                {
                  "identificador": "I",
                  "texto": "a soberania;",
                  "alineas": []
                },
                {
                  "identificador": "II",
                  "texto": "a cidadania",
                  "alineas": []
                },
                {
                  "identificador": "III",
                  "texto": "a dignidade da pessoa humana;",
                  "alineas": []
                },
          